# Notebook to Construct and Analyze Forward mu hit graphs.

### Based on: 

https://github.com/jmduarte/heptrkx-gnn-tracking/blob/master/README.md

https://github.com/jmduarte/gnn-fpga/blob/master/README.md

https://github.com/jmduarte/gnn-fpga/blob/master/gnn/GraphConstructionDev.ipynb



In [1]:
import os
import sys
import numpy as np
import pandas as pd
from collections import namedtuple
import h5py
import random 
import logging
import matplotlib.pyplot as plt
from google.colab import drive

# Mount google drive on remote Colab machine
drive.mount('/content/gdrive', force_remount=False)
sys.path.append('gdrive/My Drive/Colab Notebooks')

!ls 'gdrive/My Drive/Colab Notebooks/Data'
data_dir = 'gdrive/My Drive/Colab Notebooks/Data'

# Input and Output files and events to read
infile_mu   = data_dir+'/ntuple_SingleMuon_Endcap_9.root'
infile_pu = data_dir+'/ntuple_SingleNeutrino_PU200_63.root' 
outfile = data_dir+'/graphs.npz'
events_start=0
events_end=100


# Install uproot
!pip install uproot
import uproot

#!pip install ipynb
#!pip install import_ipynb


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
_about.txt	   graphs		  ntuple_SingleMuon_Endcap_9.root
bolsas_astro.txt   jet_images.h5	  ntuple_SingleNeutrino_PU200_63.root
bolsas_fisica.txt  ntuple_bkg_ZZ4mu.root  VBFHZZ_background.csv
events.root	   ntuple_ggH_ZZ4mu.root  VBFHZZ_signal.csv
GOOG.csv	   ntuple_qqH_ZZ4mu.root


## Define Globals and Utility Functions

In [0]:
# Event dataframe


# Event VARS from Root tree and HITVARS
VARS = ['ve_event','vh_size','vh_type', 'vh_sector', 'vh_station', 'vh_ring','vh_sim_z','vh_sim_r','vh_sim_phi', 'vh_sim_eta', 'vh_sim_theta','vh_bend','vh_sim_tp1','vh_sim_tp2']
HITVARS = ['vh_type','vh_sector', 'vh_station', 'vh_ring','vh_sim_z','vh_sim_r','vh_sim_phi', 'vh_sim_eta', 'vh_sim_theta','vh_bend','vh_sim_tp1','vh_sim_tp2']

# Event dataframe
#event_mu = pd.Series()
#event_pu = pd.Series()

# Graph is a namedtuple of (X, Ri, Ro, y) for convenience
feature_names = ['vh_sim_r', 'vh_sim_phi', 'vh_sim_z']
feature_scale = np.array([1000., 180. / 6., 1000.])
Graph = namedtuple('Graph', ['X', 'Ri', 'Ro', 'y'])

# Sparse graph uses the indices for the Ri, Ro matrices
SparseGraph = namedtuple('SparseGraph',['X', 'Ri_rows', 'Ri_cols', 'Ro_rows', 'Ro_cols', 'y'])

def graph_to_sparse(graph):
    Ri_rows, Ri_cols = graph.Ri.nonzero()
    Ro_rows, Ro_cols = graph.Ro.nonzero()
    return dict(X=graph.X, y=graph.y,
                Ri_rows=Ri_rows, Ri_cols=Ri_cols,
                Ro_rows=Ro_rows, Ro_cols=Ro_cols)

def sparse_to_graph(X, Ri_rows, Ri_cols, Ro_rows, Ro_cols, y, dtype=np.uint8):
    n_nodes, n_edges = X.shape[0], Ri_rows.shape[0]
    Ri = np.zeros((n_nodes, n_edges), dtype=dtype)
    Ro = np.zeros((n_nodes, n_edges), dtype=dtype)
    Ri[Ri_rows, Ri_cols] = 1
    Ro[Ro_rows, Ro_cols] = 1
    return Graph(X, Ri, Ro, y)



# Muon hit type
#kDT, kCSC, kRPC, kGEM, kME0 = 0, 1, 2, 3, 4

# Number of Muon PHI sectors 
nsectors=6

# MUON Detector layers 
nlayers = 12  # 5 (CSC) + 4 (RPC) + 3 (GEM)

# List of Forward Muon detector layer IDs 
csc_layers  = [ 2 , 3 , 7 , 8 , 10 ]
muon_layers = [ 0, 1, 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 ]

# EMTF hit layer number
emtf_lut = np.zeros((5,5,5), dtype=np.int32) - 99
emtf_lut[1,1,4] = 2  # ME1/1a
emtf_lut[1,1,1] = 2  # ME1/1b
emtf_lut[1,1,2] = 3  # ME1/2
emtf_lut[1,1,3] = 3  # ME1/3
emtf_lut[1,2,1] = 7  # ME2/1
emtf_lut[1,2,2] = 7  # ME2/2
emtf_lut[1,3,1] = 8  # ME3/1
emtf_lut[1,3,2] = 8  # ME3/2
emtf_lut[1,4,1] = 10 # ME4/1
emtf_lut[1,4,2] = 10 # ME4/2

emtf_lut[2,1,2] = 4  # RE1/2
emtf_lut[2,2,2] = 5  # RE2/2
emtf_lut[2,3,1] = 9  # RE3/1
emtf_lut[2,3,2] = 9  # RE3/2
emtf_lut[2,3,3] = 9  # RE3/3
emtf_lut[2,4,1] = 11 # RE4/1
emtf_lut[2,4,2] = 11 # RE4/2
emtf_lut[2,4,3] = 11 # RE4/3

emtf_lut[3,1,1] = 1  # GE1/1
emtf_lut[3,2,1] = 6  # GE2/1

emtf_lut[4,1,1] = 0  # ME0

# Get layer function
def get_layer(dtype, station, ring):
  layer = emtf_lut[dtype.astype(int),station.astype(int),ring.astype(int)] 
  return layer

# Delta phi function
def calc_dphi(phi1, phi2):
    """Computes phi2-phi1 given in range [-pi,pi]"""
    dphi = phi2 - phi1
    dphi[dphi > np.pi] -= 2*np.pi
    dphi[dphi < -np.pi] += 2*np.pi
    return dphi



# Read EVENTS function

In [0]:
# Function that reads all EVENTS from input file sand sture it in dataframes 
def readEvents():

  # Load the DATA and store selected the Root tree variables into Pandas dataframe
  upfile_mu = uproot.open(infile_mu)
  upfile_pu = uproot.open(infile_pu)
  tree_mu = upfile_mu["ntupler"]["tree"] # dictionary of NumPy arrays
  tree_pu = upfile_pu["ntupler"]["tree"] # dictionary of NumPy arrays

  #upfile_mu.keys()
  #tree_mu.show()


  # Read ROOT trees into dataframes
  df_events_mu = tree_mu.pandas.df(VARS,flatten=False, entrystart=int(events_start), entrystop=int(events_end))
  df_events_pu = tree_pu.pandas.df(VARS,flatten=False, entrystart=int(events_start), entrystop=int(events_end))
  df_muon_vp   = tree_mu.pandas.df(['vp_pt','vp_eta'], entrystart=int(events_start), entrystop=int(events_end))

  # Return events dataframes
  return df_events_mu , df_events_pu , df_muon_vp

## Get HITS Function

In [0]:
# Function that builds a HITS dataframe per event containing all real muon hits and merge it with pileup hits
def getHits(event_mu,event_pu):

 # print("event_mu = ", event_mu.head())

  # Create a HITS dataframe for a given muon event 
  hits_mu = event_mu[HITVARS]        # create a DF containing only muon hits variables
  hits_mu_list = hits_mu.values.tolist()  # evaluate jagged arrays and transform to list or arrays DF ( trick for DF of jagged arrays )
  df_hits_mu = pd.DataFrame(hits_mu_list, index=hits_mu.index) # create a dataframe from a list of arrays
  df_hits_mu = df_hits_mu.transpose() # transpose dataframe to have hit variables as columns 

  # Get only true muon hits (use generator-level matching condition)!
  df_hits_mu = df_hits_mu[(df_hits_mu['vh_sim_tp1']==0) & (df_hits_mu['vh_sim_tp2']==0)]   

  # Create a HITS only dataframe for a given pileup event 
  hits_pu = event_pu[HITVARS]         # create a DF containing only pileup hits variables
  hits_pu_list = hits_pu.values.tolist()   # evaluate jagged arrays and transform to list or arrays DF ( trick for DF of jagged arrays )
  df_hits_pu = pd.DataFrame(hits_pu_list, index=hits_pu.index)   # create a dataframe from a list of arrays
  df_hits_pu = df_hits_pu.transpose() # transpose dataframe to have hit variables as columns

  # Add "isMuon" variable to dataframes
  df_hits_mu['isMuon'] = np.ones(len(df_hits_mu))
  df_hits_pu['isMuon'] = np.zeros(len(df_hits_pu))
  
 # print("len(df_hits_mu)=",len(df_hits_mu))
 # print("df_hits_mu=",df_hits_mu.head(3))
  
 # print("len(df_hits_pu)=",len(df_hits_pu))
 # print("df_hits_pu=",df_hits_pu.head(3))

  # Concatenate MUON and PU hits dataframes into a single hits dataframe
  df_hits = pd.concat([df_hits_mu, df_hits_pu],ignore_index=True)
 
  # Add hit layer info to dataframe
  df_hits['vh_layer'] = df_hits.apply(lambda row: get_layer(row['vh_type'], row['vh_station'], row['vh_ring']), axis=1)
 
  # Filter out hits without layer information
  #df_events_mu = df_events_mu[(df_events_mu["vh_layer"]>=0)]
  #df_events_pu = df_events_pu[(df_events_pu["vh_layer"]>=0)]

 # print("len(df_hits)=",len(df_hits))
 # print("df_hits head = ", df_hits.head(3) ) 

  # Add the muon event number to the hits dataframe ( hits history )
  evtid = event_mu['ve_event'][0]        # muon event number
  df_hits["evtid"]=evtid
  # print("df_hits head=",df_hits.tail(3))

  # Return the HITS dataframe 
  return df_hits
  

## Split HITS by SECTOR Function

In [0]:
# Split hits by detector PHI sector  
def splitHitsBySector(df_hits):
     
   # List of of hits dataframes per sector 
   hits_sectors = []

   # Loop over PHI sectors
   for i in range(1,nsectors+1):

     # Select hits in this sector
     sec_hits = df_hits[df_hits['vh_sector']==i]

     # Get local sector PHI coordinate ( center hits on phi=0 )
#     if (i<=3) :
#       centered_phi = sec_hits.vh_sim_phi - i*60. +30.
#     else :
#       centered_phi = -30. + i*60. - sec_hits.vh_sim_phi 
#
#     #
#     sec_hits = sec_hits.assign(phi_local=centered_phi)
#     
#     print("isec = ",i)
#     print("sec_hits = ", sec_hits[['vh_sector','vh_type','vh_sim_phi','phi_local']])

     # Append to list 
     hits_sectors.append(sec_hits)

   # Return list of dataframes hits in each detector sector
   return hits_sectors


# Build SEGMENTS Function

In [0]:
# Function that builds SEGMENTS ( hit pairs ) using Hits dataframes. SEGMENTS are defined using hit pairs in consecutive layers
def buildSegments(df_hits):

  segments= [] # list of segments

  # Create a list of consecutive(adjacent) detector layer pairs
  layers = csc_layers
#  layers = muon_layers
  layer_pairs = [ [i,j] for i in layers for j in layers]
  adj_layer_pairs = [ x for x in layer_pairs if ( layers.index(x[0])+1 == layers.index(x[1]) ) ]
  
#  print("adj_layer_pairs=",adj_layer_pairs)

  # Group hits dataframe by layer number
  hits_groups = df_hits.groupby("vh_layer")

  # Loop over adjacent CSC layers ID pairs  
  for l1,l2 in adj_layer_pairs:

    # Group hits in consecutive layers  
    try:
      df_hits1 = hits_groups.get_group(l1)
      df_hits2 = hits_groups.get_group(l2)
    except KeyError as e:  
      logging.info('skipping empty layer: %s' % e)
#      print('skipping empty layer pair :', l1," , ",l2)
      continue
  
    # Merge hit pairs together in a single dataframe
   # df_hit_pairs = pd.merge( df_hits1.reset_index(), df_hits2.reset_index(), how='inner', on='evtid', suffixes=('_1', '_2'))
    df_hit_pairs = pd.merge( df_hits1.reset_index(), df_hits2.reset_index(), on='evtid', suffixes=('_1', '_2'))

    # Compute line segment through the hits
    dphi = calc_dphi(df_hit_pairs.vh_sim_phi_1, df_hit_pairs.vh_sim_phi_2)
    dz = df_hit_pairs.vh_sim_z_2 - df_hit_pairs.vh_sim_z_1
    dr = df_hit_pairs.vh_sim_r_2 - df_hit_pairs.vh_sim_r_1
    phi_slope = dphi / dr
    z0 = df_hit_pairs.vh_sim_z_1 - df_hit_pairs.vh_sim_r_1 * dz / dr
  
    # Discard individual hits variables and add segments( hit pairs ) variables 
    df_hit_pairs[['evtid', 'index_1', 'index_2', 'vh_layer_1', 'vh_layer_2']].assign(dphi=dphi, dz=dz, dr=dr, phi_slope=phi_slope, z0=z0)

#    print("df_hits1=",df_hits1[['evtid', 'vh_layer']])
#    print("df_hits2=",df_hits2[['evtid', 'vh_layer']])
#    print("df_hit_pairs=",df_hit_pairs[['evtid', 'index_1', 'index_2', 'vh_layer_1', 'vh_layer_2']])
#
#    print(" ")
#    print("------------------------------------------------------------------------------------------- ")
#    print(" ")

    # Filter segments according to criteria
    #sel_mask = (phi_slope.abs() < phi_slope_max) & (z0.abs() < z0_max)
    #sel_mask = (((phi_slope.abs() < phi_slope_max) & (df_hit_pairs.layer_1[0] < 5)) | ((phi_slope.abs() < phi_slope_outer_max) & (df_hit_pairs.layer_1[0] >= 5))) & (z0.abs() < z0_max)  
    #sel_mask = ((df_segments.phi_slope.abs() > phi_slope_min) &
    #                (df_segments.phi_slope.abs() < phi_slope_max) &
    #                (df_segments.z0.abs() < z0_max))
    #df_hit_pairs.assign(selected=sel_mask)

    # Create a hit pairs dataframe in segments list
    segments.append(df_hit_pairs)
 
   # End loop over segment pairs 

  # Create an all events segments dataframe from list of dataframes of event segments
  df_segments = pd.DataFrame()
  if ( len(segments) > 0 ) : 
    df_segments = pd.concat(segments, ignore_index=True)

#    print("df_segments=",df_segments[['evtid', 'index_1', 'index_2', 'vh_layer_1', 'vh_layer_2']])
#    print(" ")
#    print("------------------------------------------------------------------------------------------- ")
#    print(" ")

  # Return segments dataframe
  return df_segments


#def segment_efficiency(df_segments):
#    return (df_segments.y & df_segments.selected).sum() / df_segments.y.sum()
#
#def segment_purity(df_segment):
#    return (df_segments.y & df_segments.selected).sum() / df_segments.selected.sum()

## Build GRAPHS function

In [0]:
def buildGraph(df_hits,df_segments):

  # Get number of hits and segments in event
  n_hits = df_hits.shape[0]
  n_edges = df_segments.shape[0]

  # Prepare the GRAPH tensors( matrices )
  Ri = np.zeros((n_hits, n_edges), dtype=np.uint8)  # Adjacency matrix
  Ro = np.zeros((n_hits, n_edges), dtype=np.uint8)  # Adjacency matrix
  X = (df_hits[feature_names].values / feature_scale).astype(np.float32)  # features
  y = np.zeros(n_edges, dtype=np.float32)           # labels for SEGMENTS classification
#  y = np.zeros(n_hits, dtype=np.float32)           # labels for HITS classification

  # We have the segments given by dataframe segments
  # so we need to translate into positional indices.
  # Use a series to map hit label-index onto positional-index.
  hit_idx = pd.Series(np.arange(n_hits), index=df_hits.index)
  start_idx = hit_idx.loc[df_segments.index_1].values
  end_idx = hit_idx.loc[df_segments.index_2].values

 # print("start hit index =",start_idx)
 # print("end hit index=",end_idx)
 # print("df_segments.index_1=",df_segments.index_1)
 # print("df_segments.index_2=",df_segments.index_2)
#
#  print("n_hits=",n_hits)
#  print("n_edges=",n_edges)
#
#  print(" ")
#  print("---------------------------------------------- ")
#  print(" ")

  # Now we can fill the association matrices.
  # Note that Ri maps hits onto their incoming edges,
  # which are actually segment endings.
  Ri[end_idx, np.arange(n_edges)] = 1
  Ro[start_idx, np.arange(n_edges)] = 1
  
  # For SEGMENT classification fill the segment labels
  pid1 = df_hits.isMuon.loc[df_segments.index_1].values
  pid2 = df_hits.isMuon.loc[df_segments.index_2].values
  y[:] = [i and j for i, j in zip(pid1, pid2)]
  
 # For HITS classification fill the hits labels
 # pid1 = df_hits.isMuon.loc[df_segments.index_1].values
 # y[:] = [i for i in pid1]
  
 # print("pid1= ",pid1)
 # print("pid2= ",pid2)
  
  # Return a GRAPH (named tuple)
  return Graph(X, Ri, Ro, y)


## MAIN program


In [8]:
# Initialize the random number generator
print('Initializing random seed=1')
random.seed(1)

# Read all EVENTS into dataframes
df_events_mu , df_events_pu , df_muons_vp = readEvents()

# Get number of events in dataframes
nmu = len(df_events_mu)
npu = len(df_events_pu)

# Loop over muon events mu 
n_graphs=0
for imu, event_mu in df_events_mu.iterrows():
#for ievt, event_mu in df_events_mu.iterrows():

  # Pick a random pu event 
  ipu = random.randint(0,npu-1)
  event_pu = df_events_pu.iloc[ipu] 

  # Get HITS dataframe from merged muon and pileup events
  df_hits = getHits(event_mu, event_pu)

  # Create a list of HITS dataframe by detector Eta X Phi sector
  hits_sectors = splitHitsBySector(df_hits)

  # Loop over list of HITS per sector
  for isec, df_hits_sec in enumerate(hits_sectors): 

    # Skip empty sectors
    if (len(df_hits_sec)==0):
#      print("Skipping empty sector",isec)
      continue
      
    # Build SEGMENTS ( hits pairs in adjacent layers )
    df_segments = buildSegments(df_hits_sec)

    # Skip empty segments
    if ( len(df_segments)==0 ):
#      print("Skipping empty segments",isec)
      continue 

    # Build GRAPH
    graph = buildGraph(df_hits_sec,df_segments)

    print('Graph shape:', graph.X.shape , graph.Ri.shape , graph.Ro.shape , graph.y.shape)
   # print("Graph = ", graph)
   # print(" ")
   # print(" ---------------------------------------------------- ")
   # print(" ")

    # Convert GRAPH to a dictionary to save as NPZ
    gdic = dict(X=graph.X, Ri=graph.Ri, Ro=graph.Ro, y=graph.y)
    
    # Save GRAPH dictionary to a NPZ file
    n_graphs+=1
    outfile = data_dir+'/graphs/graph_'+str(n_graphs)+'.npz' 
    np.savez(outfile,**gdic)

    # Convert to SPARSE GRAPHS and save to NPZ file
#    sparse_graph = graph_to_sparse(graph) 
#    outfile = data_dir+'/graphs/sgraph_'+str(imu)+'.npz' 
#    np.savez(outfile,**sparse_graph)



    # End loop over hits per sector
  # End loop over events



Initializing random seed=1
Graph shape: (14, 3) (14, 5) (14, 5) (5,)
Graph shape: (16, 3) (16, 14) (16, 14) (14,)
Graph shape: (12, 3) (12, 3) (12, 3) (3,)
Graph shape: (18, 3) (18, 8) (18, 8) (8,)
Graph shape: (16, 3) (16, 11) (16, 11) (11,)
Graph shape: (9, 3) (9, 1) (9, 1) (1,)
Graph shape: (6, 3) (6, 1) (6, 1) (1,)
Graph shape: (12, 3) (12, 4) (12, 4) (4,)
Graph shape: (17, 3) (17, 7) (17, 7) (7,)
Graph shape: (13, 3) (13, 6) (13, 6) (6,)
Graph shape: (10, 3) (10, 5) (10, 5) (5,)
Graph shape: (8, 3) (8, 2) (8, 2) (2,)
Graph shape: (10, 3) (10, 2) (10, 2) (2,)
Graph shape: (24, 3) (24, 11) (24, 11) (11,)
Graph shape: (8, 3) (8, 1) (8, 1) (1,)
Graph shape: (22, 3) (22, 2) (22, 2) (2,)
Graph shape: (21, 3) (21, 6) (21, 6) (6,)
Graph shape: (5, 3) (5, 1) (5, 1) (1,)
Graph shape: (24, 3) (24, 6) (24, 6) (6,)
Graph shape: (13, 3) (13, 7) (13, 7) (7,)
Graph shape: (10, 3) (10, 2) (10, 2) (2,)
Graph shape: (20, 3) (20, 25) (20, 25) (25,)
Graph shape: (9, 3) (9, 4) (9, 4) (4,)
Graph shape: 